#### This section covers the process of getting the 500bp upstream and downstream sequences from the TSS of each gene

In [79]:
df = pd.read_csv("C:/Users/muell/OneDrive/Documents/ML_FinalProject/final_gene_expression_data.csv")

In [80]:
# Create a mapping of cleaned IDs to original IDs
df['cleaned_gene_ids'] = df['Name'].str.split('.').str[0]
id_mapping = df[['cleaned_gene_ids', 'Name']].drop_duplicates()

# Save the mapping
id_mapping.to_csv("id_mapping.csv", index=False)

# Save the cleaned gene IDs
id_mapping['cleaned_gene_ids'].to_csv("unique_gene_ids.txt", index=False, header=False)

In [91]:
id_mapping.shape

(22394, 2)

In [86]:
import pandas as pd
import re

# GTF file details
gtf_file = "Homo_sapiens.GRCh38.113.gtf"
gtf_cols = ["chromosome", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]

# Specify column data types
gtf_dtypes = {
    "chromosome": "string",
    "source": "string",
    "feature": "string",
    "start": "Int64", 
    "end": "Int64",
    "score": "string",
    "strand": "string",
    "frame": "string",
    "attribute": "string",
}

# Load the GTF file with specified dtypes
gtf = pd.read_csv(gtf_file, sep="\t", comment="#", names=gtf_cols, dtype=gtf_dtypes, low_memory=False)

In [87]:
# Filter for "gene" features only
genes = gtf[gtf["feature"] == "gene"].copy()

# Define a function for safe regex extraction
def extract_gene_id(attribute):
    match = re.search(r'gene_id "([^"]+)"', attribute)
    return match.group(1) if match else None

# Extract the gene_id from the attribute column
genes["gene_id"] = genes["attribute"].apply(extract_gene_id)

# Verify the resulting DataFrame
print(genes.head())


    chromosome          source feature     start       end score strand frame  \
0            1  ensembl_havana    gene   3069168   3438621     .      +     .   
220          1          havana    gene   5301928   5307394     .      -     .   
226          1  ensembl_havana    gene   2403964   2413797     .      -     .   
346          1          havana    gene   5492978   5494674     .      +     .   
349          1          havana    gene  10054445  10054781     .      -     .   

                                             attribute          gene_id  
0    gene_id "ENSG00000142611"; gene_version "17"; ...  ENSG00000142611  
220  gene_id "ENSG00000284616"; gene_version "1"; g...  ENSG00000284616  
226  gene_id "ENSG00000157911"; gene_version "11"; ...  ENSG00000157911  
346  gene_id "ENSG00000260972"; gene_version "1"; g...  ENSG00000260972  
349  gene_id "ENSG00000224340"; gene_version "1"; g...  ENSG00000224340  


In [88]:
# Load the unique gene IDs
unique_gene_ids = pd.read_csv("unique_gene_ids.txt", header=None, names=["gene_id"])

# Filter the GTF genes to include only those in the unique gene list
filtered_genes = genes[genes["gene_id"].isin(unique_gene_ids["gene_id"])]

In [96]:
filtered_genes.head()

,chromosome,source,feature,start,end,score,strand,frame,attribute,gene_id,tss_start,tss_end
226,1,ensembl_havana,gene,2403964,2413797,.,-,.,"gene_id ""ENSG00000157911""; gene_version ""11""; ...",ENSG00000157911,2413297,2414297
355,1,ensembl_havana,gene,10472288,10630758,.,+,.,"gene_id ""ENSG00000142655""; gene_version ""13""; ...",ENSG00000142655,10471788,10472788
444,1,ensembl_havana,gene,2425980,2505532,.,+,.,"gene_id ""ENSG00000149527""; gene_version ""18""; ...",ENSG00000149527,2425480,2426480
676,1,ensembl_havana,gene,9292894,9369532,.,+,.,"gene_id ""ENSG00000171621""; gene_version ""14""; ...",ENSG00000171621,9292394,9293394
721,1,havana,gene,2492300,2493258,.,-,.,"gene_id ""ENSG00000224387""; gene_version ""1""; g...",ENSG00000224387,2492758,2493758


In [92]:
# Save to a new GTF-like file
filtered_genes.to_csv("filtered_genes.gtf", sep="\t", index=False, header=False)

In [97]:
# Select the required columns for BED format
bed_df = filtered_genes[["chromosome", "tss_start", "tss_end", "gene_id", "score", "strand"]]

# Save to a BED file
bed_df.to_csv("gtex_transcripts.bed", sep="\t", index=False, header=False)

In [140]:
import pandas as pd

# Load gene expression data
expression_df = pd.read_csv("final_gene_expression_data.csv")

# Load the mapping file (gene_id to transcript_id)
mapping_df = pd.read_csv("filtered_mapping.txt", sep="\t", names=["gene_id", "transcript_id"])

# Load the sequences (transcript_id and sequence)
sequences_df = pd.read_csv("gtex_sequences.csv", header=None, names=["transcript_id", "sequence"])

In [141]:
expression_df.head()

,Name,Description,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,ENSG00000227232.5,WASH7P,-0.123846,-0.013572,-0.002610,-0.008449,-0.000559,-0.008659,-0.001629,0.001415,0.002948,0.000726
1,ENSG00000233750.3,CICP27,-0.127444,-0.014391,-0.002761,-0.008784,-0.001466,-0.009513,-0.002162,0.001299,0.003319,0.000545
2,ENSG00000268903.1,ENSG00000268903,0.073242,-0.012280,-0.001378,0.065349,0.011552,0.034967,-0.050337,-0.033125,-0.027376,0.001724
3,ENSG00000269981.1,ENSG00000269981,0.197083,-0.010883,0.000580,0.087887,-0.012004,0.052083,-0.053531,-0.069170,-0.050675,-0.023379
4,ENSG00000279928.2,DDX11L17,-0.128000,-0.014400,-0.002366,-0.009148,-0.001414,-0.009585,-0.001641,0.001489,0.003598,0.000606


In [142]:
mapping_df.head()

,gene_id,transcript_id
0,ENSG00000157911,ENST00000288774
1,ENSG00000157911,ENST00000447513
2,ENSG00000157911,ENST00000650293
3,ENSG00000157911,ENST00000507596
4,ENSG00000157911,ENST00000510434


In [143]:
sequences_df.head()

,transcript_id,sequence
0,ENSG00000157911::1:2413297-2414297(-),GGTGCAGATGAGAAGAGTGGGGAAAGCAGTCCTGAGCCAGGACAGC...
1,ENSG00000142655::1:10471788-10472788(+),AGTGCAAGGCCAAGCAGCTGGTAAAGGTATGGGGCCGGGCTCGGAA...
2,ENSG00000149527::1:2425480-2426480(+),GCAGGAGTGGGGTTGCTGGATCTGCTCTGCTGGTTTTTAGGTGGCA...
3,ENSG00000171621::1:9292394-9293394(+),AAACGCGAGCGGCCGAAACAGCGCTGCAGGAGGCTCCCTGGACCCC...
4,ENSG00000224387::1:2492758-2493758(-),ACGCGTTGGAAATCAGGGCACGTCGCGTCCGCAGGCCCCATGCCAG...


In [144]:
expression_df['cleaned_gene_id'] = expression_df['Name'].str.split('.').str[0]

In [145]:
# Clean transcript_id in sequences_df
sequences_df['gene_id'] = sequences_df['transcript_id'].str.split('::').str[0]

In [146]:
sequences_df.head()

,transcript_id,sequence,gene_id
0,ENSG00000157911::1:2413297-2414297(-),GGTGCAGATGAGAAGAGTGGGGAAAGCAGTCCTGAGCCAGGACAGC...,ENSG00000157911
1,ENSG00000142655::1:10471788-10472788(+),AGTGCAAGGCCAAGCAGCTGGTAAAGGTATGGGGCCGGGCTCGGAA...,ENSG00000142655
2,ENSG00000149527::1:2425480-2426480(+),GCAGGAGTGGGGTTGCTGGATCTGCTCTGCTGGTTTTTAGGTGGCA...,ENSG00000149527
3,ENSG00000171621::1:9292394-9293394(+),AAACGCGAGCGGCCGAAACAGCGCTGCAGGAGGCTCCCTGGACCCC...,ENSG00000171621
4,ENSG00000224387::1:2492758-2493758(-),ACGCGTTGGAAATCAGGGCACGTCGCGTCCGCAGGCCCCATGCCAG...,ENSG00000224387


In [147]:
# Merge expression_df with mapping_df on gene_id
expression_with_transcripts = pd.merge(expression_df, mapping_df, left_on="cleaned_gene_id", right_on="gene_id", how="inner")

In [148]:
expression_with_transcripts.head()

,Name,Description,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,cleaned_gene_id,gene_id,transcript_id
0,ENSG00000227232.5,WASH7P,-0.123846,-0.013572,-0.002610,-0.008449,-0.000559,-0.008659,-0.001629,0.001415,0.002948,0.000726,ENSG00000227232,ENSG00000227232,ENST00000488147
1,ENSG00000233750.3,CICP27,-0.127444,-0.014391,-0.002761,-0.008784,-0.001466,-0.009513,-0.002162,0.001299,0.003319,0.000545,ENSG00000233750,ENSG00000233750,ENST00000442987
2,ENSG00000268903.1,ENSG00000268903,0.073242,-0.012280,-0.001378,0.065349,0.011552,0.034967,-0.050337,-0.033125,-0.027376,0.001724,ENSG00000268903,ENSG00000268903,ENST00000494149
3,ENSG00000269981.1,ENSG00000269981,0.197083,-0.010883,0.000580,0.087887,-0.012004,0.052083,-0.053531,-0.069170,-0.050675,-0.023379,ENSG00000269981,ENSG00000269981,ENST00000595919
4,ENSG00000279928.2,DDX11L17,-0.128000,-0.014400,-0.002366,-0.009148,-0.001414,-0.009585,-0.001641,0.001489,0.003598,0.000606,ENSG00000279928,ENSG00000279928,ENST00000624431


In [151]:
final_merged_df = pd.merge(expression_with_transcripts, sequences_df, on="gene_id", how="inner")

In [157]:
final_merged_df

,Name,Description,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,cleaned_gene_id,gene_id,transcript_id_x,transcript_id_y,sequence
0,ENSG00000227232.5,WASH7P,-0.123846,-0.013572,-0.002610,-0.008449,-0.000559,-0.008659,-0.001629,0.001415,0.002948,0.000726,ENSG00000227232,ENSG00000227232,ENST00000488147,ENSG00000227232::1:24386-25386(-),AACCCGTGTGTGCACATTTTTTGTGCTTTTCCAGAACTGTGTACCA...
1,ENSG00000233750.3,CICP27,-0.127444,-0.014391,-0.002761,-0.008784,-0.001466,-0.009513,-0.002162,0.001299,0.003319,0.000545,ENSG00000233750,ENSG00000233750,ENST00000442987,ENSG00000233750::1:130525-131525(+),GTCAAAAGTTAAAGAAACTTTGCAGGTCAGGACAGAATCGAATAAT...
2,ENSG00000268903.1,ENSG00000268903,0.073242,-0.012280,-0.001378,0.065349,0.011552,0.034967,-0.050337,-0.033125,-0.027376,0.001724,ENSG00000268903,ENSG00000268903,ENST00000494149,ENSG00000268903::1:135395-136395(-),TCACACCGGCCCCTCCCACGCTGAGAGAGGTCAGTGTGAGCCCTTG...
3,ENSG00000269981.1,ENSG00000269981,0.197083,-0.010883,0.000580,0.087887,-0.012004,0.052083,-0.053531,-0.069170,-0.050675,-0.023379,ENSG00000269981,ENSG00000269981,ENST00000595919,ENSG00000269981::1:137465-138465(-),CAAATCAGGCTTTTGCCCAACTTCTGTCTACTGTCGGACTCTACAG...
4,ENSG00000279928.2,DDX11L17,-0.128000,-0.014400,-0.002366,-0.009148,-0.001414,-0.009585,-0.001641,0.001489,0.003598,0.000606,ENSG00000279928,ENSG00000279928,ENST00000624431,ENSG00000279928::1:182196-183196(+),CCCTGGAGATTCTTATTAGTGATTTGGGCTGGGGCCTGGCCATGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193942,ENSG00000198695.2,MT-ND6,1.288162,0.257889,-0.012438,0.160438,0.039930,0.555881,0.393585,-0.420538,0.035647,0.134114,ENSG00000198695,ENSG00000198695,ENST00000361681,ENSG00000198695::MT:14173-15173(-),CCCCTCAGAATGATATTTGGCCTCACGGGAGGACATAGCCTATGAA...
193943,ENSG00000210194.1,MT-TE,-0.124354,-0.013892,-0.002712,-0.008277,-0.001226,-0.008314,-0.000863,0.000342,0.003963,0.001268,ENSG00000210194,ENSG00000210194,ENST00000387459,ENSG00000210194::MT:14242-15242(-),CTCAGATTCATTGAACTAGGTCTGTCCCAATGTATGGGATGGCGGA...
193944,ENSG00000198727.2,MT-CYB,3.957295,2.872498,0.318762,0.160265,0.302238,1.740722,0.944125,-0.666716,0.313799,0.260474,ENSG00000198727,ENSG00000198727,ENST00000361789,ENSG00000198727::MT:14247-15247(+),CGCACCAATAGGATCCTCCCGAATCAACCCTGACCCCTCTCCTTCA...
193945,ENSG00000210195.2,MT-TT,-0.128244,-0.014401,-0.002581,-0.009073,-0.001497,-0.009844,-0.001851,0.001706,0.003488,0.000751,ENSG00000210195,ENSG00000210195,ENST00000387460,ENSG00000210195::MT:15388-16388(+),TCCGATAAAATCACCTTCCACCCTTACTACACAATCAAAGACGCCC...


In [159]:
# Check how many unique gene IDs are duplicated
print(f"Total unique gene IDs: {final_merged_df['gene_id'].nunique()}")
print(f"Total rows: {final_merged_df.shape[0]}")

# Display some duplicate examples
duplicates = final_merged_df[final_merged_df['gene_id'].duplicated(keep=False)]
print(duplicates.head())

Total unique gene IDs: 22114
Total rows: 193947
                  Name Description       PC1       PC2       PC3       PC4  \
17  ENSG00000237491.10   LINC01409 -0.128374 -0.012855 -0.002552 -0.009135   
18  ENSG00000237491.10   LINC01409 -0.128374 -0.012855 -0.002552 -0.009135   
19  ENSG00000237491.10   LINC01409 -0.128374 -0.012855 -0.002552 -0.009135   
20  ENSG00000237491.10   LINC01409 -0.128374 -0.012855 -0.002552 -0.009135   
21  ENSG00000237491.10   LINC01409 -0.128374 -0.012855 -0.002552 -0.009135   

         PC5       PC6       PC7       PC8       PC9      PC10  \
17 -0.001463 -0.009729 -0.001872  0.001706  0.003546  0.000692   
18 -0.001463 -0.009729 -0.001872  0.001706  0.003546  0.000692   
19 -0.001463 -0.009729 -0.001872  0.001706  0.003546  0.000692   
20 -0.001463 -0.009729 -0.001872  0.001706  0.003546  0.000692   
21 -0.001463 -0.009729 -0.001872  0.001706  0.003546  0.000692   

    cleaned_gene_id          gene_id  transcript_id_x  \
17  ENSG00000237491  ENSG0000

In [160]:
# Drop duplicates, keeping the first occurrence of each gene_id
unique_final_df = final_merged_df.drop_duplicates(subset='gene_id', keep='first')

# Verify the results
print(f"Shape after removing duplicates: {unique_final_df.shape}")

Shape after removing duplicates: (22114, 17)


In [ ]:
unique_final_df = unique_final_df.drop(columns=["cleaned_gene_id"])

In [164]:
unique_final_df = unique_final_df.drop(columns=["gene_id"])

In [166]:
unique_final_df.to_csv("final_expression_with_sequences.csv", index=False)

#### The section below represents commands run to process some of the data and to run BEDtools in conjunction with this notebook

In [ ]:
'''

Getting Genomic Sequences from Reference Genome (500bp upstream and downstream of each TSS):

# Prepare a list of Ensembl IDs (one per line) from the GTEx data
cut -f1 final_gene_expression_data.csv | sed 's/\\..*$//' > gtex_transcript_ids.txt

# Extract Only the First Column, Remove the Header Line, and Remove Version Numbers
cut -d',' -f1 gtex_transcript_ids.txt | tail -n +2 | sed 's/\.[0-9]*$//' > gtex_transcript_ids_no_version.txt

# Use awk to extract the gene_id and transcript_id pairs from the GTF file
awk '$3 == "transcript" {match($0, /gene_id "([^"]+)"/, g); match($0, /transcript_id "([^"]+)"/, t); if (g[1] && t[1]) print g[1] "\t" t[1]}' Homo_sapiens.GRCh38.113.gtf > gene_to_transcript_map.txt

# Filter the gene_to_transcript_map.txt file to include only the gene IDs present in the gtex_transcript_ids_clean.txt file
grep -Ff gtex_transcript_ids_no_version.txt gene_to_transcript_map.txt > filtered_gene_to_transcript_map.txt

# Extract the transcript IDs
cut -f2 filtered_gene_to_transcript_map.txt > transcript_ids.txt

#Filter the GTF file:
awk '$3 == "transcript"' Homo_sapiens.GRCh38.113.gtf | grep -Ff transcript_ids.txt > filtered.gtf

# Convert the filtered GTF file to a BED file while extending regions by 500 base pairs upstream and downstream
awk 'BEGIN {OFS="\t"} {match($0, /transcript_id "([^"]+)"/, t); if (t[1]) print $1, $4-1, $5, t[1], ".", $7}' filtered_genes.gtf > gtex_transcripts.bed

# Use BEDTools to extract sequences from the reference genome
bedtools getfasta -fi Homo_sapiens.GRCh38.dna.primary_assembly.fa -bed gtex_transcripts.bed -fo gtex_sequences.fa -name -s

# Convert fasta file to csv format
awk 'BEGIN {OFS=","} /^>/ {if (seq) print id, seq; id=substr($1, 2); seq=""} !/^>/ {seq=seq $0} END {print id, seq}' gtex_sequences.fa > gtex_sequences.csv


'''